In [133]:
import pandas as pd
import numpy as np
from collections import Counter
import re
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split,GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from xgboost import XGBRegressor
import pickle
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
from sklearn.exceptions import NotFittedError
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
# Comment this if the data visualisations doesn't work on your side
%matplotlib inline

plt.style.use('bmh')

In [134]:
train = pd.read_excel('C:/Users/Desktop/ML_Hack/Participants_Data_Used_Cars/Data_Train.xlsx')
test = pd.read_excel('C:/Users/Desktop/ML_Hack/Participants_Data_Used_Cars/Data_Test.xlsx')

In [135]:
train['Name']=train['Name'].str.strip()
train['Name']=train['Name'].str.upper()
test['Name']=test['Name'].str.strip()
test['Name']=test['Name'].str.upper()


train['Location']=train['Location'].str.strip()
train['Location']=train['Location'].str.upper()
test['Location']=test['Location'].str.strip()
test['Location']=test['Location'].str.upper()

train['Fuel_Type']=train['Fuel_Type'].str.strip()
train['Fuel_Type']=train['Fuel_Type'].str.upper()
test['Fuel_Type']=test['Fuel_Type'].str.strip()
test['Fuel_Type']=test['Fuel_Type'].str.upper()


train['Transmission']=train['Transmission'].str.strip()
train['Transmission']=train['Transmission'].str.upper()
test['Transmission']=test['Transmission'].str.strip()
test['Transmission']=test['Transmission'].str.upper()


train['Owner_Type']=train['Owner_Type'].str.strip()
train['Owner_Type']=train['Owner_Type'].str.upper()
test['Owner_Type']=test['Owner_Type'].str.strip()
test['Owner_Type']=test['Owner_Type'].str.upper()



train['Mileage']=train['Mileage'].str.strip()
train['Mileage']=train['Mileage'].str.upper()
test['Mileage']=test['Mileage'].str.strip()
test['Mileage']=test['Mileage'].str.upper()



train['Engine']=train['Engine'].str.strip()
train['Engine']=train['Engine'].str.upper()
test['Engine']=test['Engine'].str.strip()
test['Engine']=test['Engine'].str.upper()



train['Power']=train['Power'].str.strip()
train['Power']=train['Power'].str.upper()
test['Power']=test['Power'].str.strip()
test['Power']=test['Power'].str.upper()



train['New_Price']=train['New_Price'].str.strip()
train['New_Price']=train['New_Price'].str.upper()
test['New_Price']=test['New_Price'].str.strip()
test['New_Price']=test['New_Price'].str.upper()

In [136]:
#Outlier Treatment:

#train=train[(train['Kilometers_Driven']  > 999) & (train['Kilometers_Driven'] < 6500000)]
#train = train.reset_index(drop=True)

In [137]:
train.shape

(6019, 13)

In [138]:
tg=train[['Price']]
del train['Price']

In [139]:
all_data=pd.concat([train,test])
all_data = all_data.reset_index(drop=True)

In [140]:
#all_data['CR_fg']=np.where(all_data['New_Price'].str.contains('CR',na=False),1,0)
#all_data['LAKH_fg']=np.where(all_data['New_Price'].str.contains('LAKH',na=False),1,0)
#all_data['KMKG_fg']=np.where(all_data['Mileage'].str.contains('KM/KG',na=False),1,0)
#all_data['KMPL_fg']=np.where(all_data['Mileage'].str.contains('KMPL',na=False),1,0)
#all_data['NULL_fg']=np.where(all_data['Power'].str.contains('NULL',na=False),1,0)

In [141]:
all_data.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price
0,MARUTI WAGON R LXI CNG,MUMBAI,2010,72000,CNG,MANUAL,FIRST,26.6 KM/KG,998 CC,58.16 BHP,5.0,NaN
1,HYUNDAI CRETA 1.6 CRDI SX OPTION,PUNE,2015,41000,DIESEL,MANUAL,FIRST,19.67 KMPL,1582 CC,126.2 BHP,5.0,NaN
2,HONDA JAZZ V,CHENNAI,2011,46000,PETROL,MANUAL,FIRST,18.2 KMPL,1199 CC,88.7 BHP,5.0,8.61 LAKH
3,MARUTI ERTIGA VDI,CHENNAI,2012,87000,DIESEL,MANUAL,FIRST,20.77 KMPL,1248 CC,88.76 BHP,7.0,NaN
4,AUDI A4 NEW 2.0 TDI MULTITRONIC,COIMBATORE,2013,40670,DIESEL,AUTOMATIC,SECOND,15.2 KMPL,1968 CC,140.8 BHP,5.0,NaN


In [142]:
all_data.shape

(7253, 12)

In [143]:
names1 = list(all_data['Name'])
for i in range(len(names1)) :
    try:
       names1[i] = names1[i].split(" ")[0].strip().upper()
    except :
       pass 

In [144]:
names2 = list(all_data['Name'])
for i in range(len(names2)) :
    try:
       names2[i] = names2[i].split(" ")[1].strip().upper()
    except :
       pass 

In [145]:
car_data = {}
car_data['Brand'] = names1
car_data['Sub_Brand'] = names2
car_data = pd.DataFrame(car_data)

In [146]:
all_data['Name']=all_data['Name'].str.upper()
all_data['Location']=all_data['Location'].str.upper()
all_data['Fuel_Type']=all_data['Fuel_Type'].str.upper()
all_data['Transmission']=all_data['Transmission'].str.upper()
all_data['Owner_Type']=all_data['Owner_Type'].str.upper()

In [147]:
#Cleaning Engine, Power and Mileage

all_data['Engine']=all_data['Engine'].str.replace('[A-Za-z]','')
all_data['Engine'] = all_data['Engine'].str.strip()
all_data['Engine'] = all_data['Engine'].apply(float)

all_data['Power']=all_data['Power'].str.replace('[A-Za-z]','')
all_data['Power'] = all_data['Power'].str.strip()
all_data.Power = all_data.Power.fillna('')


all_data['Mileage']=all_data['Mileage'].str.replace('[A-Za-z,/]','')
all_data['Mileage'] = all_data['Mileage'].str.strip()
all_data['Mileage'] = all_data['Mileage'].apply(float)

In [148]:
Power = list(all_data['Power'])

for i in range(len(Power)) :
    try:
       Power[i] = float(Power[i])
    except :
       Power[i] = np.nan
    

num_data = {}
num_data['Power'] = Power
num_data = pd.DataFrame(num_data)

In [149]:
del all_data['Power']
all_data=pd.concat([all_data,num_data,car_data], axis=1)

In [150]:
new = all_data['New_Price'].str.split(" ", n = 1, expand = True) 
all_data["Price"]= new[0] 
all_data['Price'] = all_data['Price'].apply(float)
all_data["Currency"]= new[1]

all_data['Cr']=np.where(all_data['Currency'].str.contains('Cr'),1,0)

In [151]:
all_data["Final_Price"]= np.where(all_data['Cr']==1, all_data["Price"]*100, all_data["Price"])

In [152]:
del all_data['New_Price']
del all_data['Price']
del all_data['Currency']
del all_data['Cr']

In [153]:
all_data['History']=2019 - all_data['Year']

In [154]:
all_data['Year'] = all_data['Year'].apply(str)

In [155]:
t1=all_data.isnull().sum()
t1=pd.DataFrame(t1)
t1.columns=['Values']
t1[t1.Values>0]

,Values
Mileage,2
Engine,46
Seats,53
Power,175
Final_Price,6247


In [156]:
impute_val=all_data.groupby(['Brand','Sub_Brand','Fuel_Type', 'Transmission'], as_index=False)[['Seats', 'Power','Engine','Mileage']].median()
impute_val.columns=['Brand','Sub_Brand','Fuel_Type', 'Transmission','Seats1','Power1','Engine1','Mileage1']

In [157]:
all_data_v2=pd.merge(all_data, impute_val, on=['Brand','Sub_Brand','Fuel_Type', 'Transmission'], how='left')

In [158]:
all_data_v2["Seats_v1"]= np.where(all_data_v2['Seats']> 0, all_data_v2['Seats'], all_data_v2['Seats1'])
all_data_v2["Power_v1"]= np.where(all_data_v2['Power']> 0, all_data_v2['Power'], all_data_v2['Power1'])
all_data_v2["Engine_v1"]= np.where(all_data_v2['Engine']> 0, all_data_v2['Engine'], all_data_v2['Engine1'])
all_data_v2["Mileage_v1"]= np.where(all_data_v2['Mileage']> 0, all_data_v2['Mileage'], all_data_v2['Mileage1'])

In [159]:
del all_data_v2['Seats']
del all_data_v2['Seats1']

del all_data_v2['Power']
del all_data_v2['Power1']

del all_data_v2['Mileage']
del all_data_v2['Mileage1']

del all_data_v2['Engine']
del all_data_v2['Engine1']

In [160]:
all_data_v2['Seats_v1']=all_data_v2['Seats_v1'].fillna(all_data_v2['Seats_v1'].median())
all_data_v2['Power_v1']=all_data_v2['Power_v1'].fillna(all_data_v2['Power_v1'].median())
all_data_v2['Engine_v1']=all_data_v2['Engine_v1'].fillna(all_data_v2['Engine_v1'].median())
all_data_v2['Mileage_v1']=all_data_v2['Mileage_v1'].fillna(all_data_v2['Mileage_v1'].median())


all_data_v2['Final_Price']=all_data_v2['Final_Price'].fillna(-999)

#all_data_v2.fillna(0, inplace = True)

In [161]:
cat_cols=all_data_v2.columns[all_data_v2.dtypes=='object'].tolist()
print(cat_cols)

num_cols=all_data_v2.columns[all_data_v2.dtypes!='object'].tolist()
print(num_cols)

['Name', 'Location', 'Year', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Brand', 'Sub_Brand']
['Kilometers_Driven', 'Final_Price', 'History', 'Seats_v1', 'Power_v1', 'Engine_v1', 'Mileage_v1']


In [162]:
# Additional Removal Part:
train1_dup1=all_data_v2[:6019]
test1_dup1=all_data_v2[6019:7253]

train1_dup2=pd.concat([train1_dup1,tg], axis=1)

train1_dup2['flag']=train1_dup2['Fuel_Type']+train1_dup2['Location']+train1_dup2['Name']+train1_dup2['Owner_Type']+train1_dup2['Transmission']+train1_dup2['Year']+train1_dup2['Brand']+train1_dup2['Sub_Brand']+train1_dup2['Kilometers_Driven'].astype(str)+train1_dup2['Final_Price'].astype(str)+ train1_dup2['History'].astype(str)+train1_dup2['Seats_v1'].astype(str)+train1_dup2['Power_v1'].astype(str)+train1_dup2['Engine_v1'].astype(str)+train1_dup2['Mileage_v1'].astype(str)

dup1=train1_dup2[train1_dup2.flag.duplicated()]
dup2=dup1.drop_duplicates(subset ="flag",keep='first')
dup2=dup2[['flag']]

dup_inner=pd.merge(dup2, train1_dup2, on='flag', how='inner')

dup_inner2=dup_inner.groupby('flag', as_index=False).agg({"Price": "mean"})
dup_inner2.columns = ['flag', 'Price1']

dup_inner3=pd.merge(dup_inner,dup_inner2, on='flag', how='inner')

dup_inner4=dup_inner3.drop_duplicates(subset ="flag",keep='first') 

del dup_inner4['flag']
del dup_inner4['Price']

dup_inner4 = dup_inner4.rename(columns = {"Price1": "Price"})

train1_dup3=train1_dup2.drop_duplicates(subset ="flag",keep = False)

del train1_dup3['flag']

train1_dup4=pd.concat([train1_dup3,dup_inner4])
train1_dup4 = train1_dup4.reset_index(drop=True)

train1_dup4.shape

(6001, 16)

In [163]:
tg=train1_dup4[['Price']]
tg["Price1"] = np.log1p(tg["Price"].astype(np.float64))

target=tg.Price1#Change
del train1_dup4['Price']

all_data_v2=pd.concat([train1_dup4,test1_dup1])
all_data_v2 = all_data_v2.reset_index(drop=True)

C:\Users\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [164]:
all_data_v2.shape


(7235, 15)

In [165]:
all_data_v2.describe()

,Kilometers_Driven,Final_Price,History,Seats_v1,Power_v1,Engine_v1,Mileage_v1
count,7.235000e+03,7235.000000,7235.000000,7235.000000,7235.000000,7235.000000,7235.000000
mean,5.873222e+04,-857.326511,5.633725,5.278784,112.134200,1615.251831,18.294951
std,8.451807e+04,352.633886,3.255190,0.807305,53.309144,594.226306,4.204655
min,1.710000e+02,-999.000000,0.000000,2.000000,34.200000,72.000000,0.000000
25%,3.400000e+04,-999.000000,3.000000,5.000000,74.000000,1198.000000,15.290000
50%,5.350000e+04,-999.000000,5.000000,5.000000,92.850000,1493.000000,18.160000
75%,7.300000e+04,-999.000000,8.000000,5.000000,138.030000,1968.000000,21.100000
max,6.500000e+06,99.920000,23.000000,10.000000,616.000000,5998.000000,33.540000


In [166]:
class Encoding(BaseEstimator):
    categorical_columns = None
    return_df = False
    random_state = 30
    threshold = 50

    def __init__(self):
        pass

    def convert_input(self, X):
        if not isinstance(X, pd.DataFrame):
            if isinstance(X, list):
                X = pd.DataFrame(np.array(X))
            elif isinstance(X, (np.generic, np.ndarray, pd.Series)):
                X = pd.DataFrame(X)
            else:
                raise ValueError('Unexpected input type: %s' % (str(type(X))))
            X = X.apply(lambda x: pd.to_numeric(x, errors='ignore'))
        x = X.copy(deep = True)
        return x

    def get_categorical_columns(self, X):
        return X.select_dtypes(include=['object', 'category']).columns.tolist()

    def get_numerical_columns(self,X):
        temp_x=X[X.columns[X.nunique()<=self.threshold]]
        col_names=temp_x.columns[temp_x.dtypes!='object']
        return col_names

    def apply_encoding(self, X_in, encoding_dict):
        X = self.convert_input(X_in)
        for col in self.categorical_columns:
            if col in encoding_dict:
                freq_dict = encoding_dict[col]
                X[col] = X[col].apply(lambda x: freq_dict[x] if x  in freq_dict else np.nan)
        return X

    def create_encoding_dict(self, X, y):
        return {}

    def fit(self, X, y=None):
        if X is None:
            raise ValueError("Input array is required to call fit method!")
        X = self.convert_input(X)
        self.encoding_dict = self.create_encoding_dict(X, y)
        return self

    def transform(self, X):
        df = self.apply_encoding(X, self.encoding_dict)
        if self.return_df:
            return df
        else:
            return df.values

    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)

    def inverse_transform(self, X):
        X = self.convert_input(X)
        for col in self.categorical_columns:
            freq_dict = self.encoding_dict[col]
            for key, val in freq_dict.iteritems():
                X.loc[X[col] == val, col] = key
        if self.return_df:
            return X
        else:
            return X.values

In [167]:
class FreqeuncyEncoding(Encoding):
    '''
    class to perform FreqeuncyEncoding on Categorical Variables
    Initialization Variabes:
    categorical_columns: list of categorical columns from the dataframe
    or list of indexes of caategorical columns for numpy ndarray
    return_df: boolean
        if True: returns pandas dataframe on transformation
        else: return numpy ndarray
    '''
    def __init__(self, categorical_columns = None, return_df = False):
        self.categorical_columns = categorical_columns
        self.return_df = return_df

    def create_encoding_dict(self, X, y):
        encoding_dict = {}
        if self.categorical_columns is None:
            self.categorical_columns = self.get_categorical_columns(X)
        for col in self.categorical_columns:
            encoding_dict.update({col: X[col].value_counts(normalize = True).to_dict()})
        return encoding_dict


fe=FreqeuncyEncoding(categorical_columns=cat_cols,return_df=True)
all_data_v3=fe.fit_transform(all_data_v2)

In [168]:
titles = list(all_data_v2['Location'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  11


Unique Titles:
 ['MUMBAI' 'PUNE' 'CHENNAI' 'COIMBATORE' 'HYDERABAD' 'JAIPUR' 'KOCHI'
 'KOLKATA' 'DELHI' 'BANGALORE' 'AHMEDABAD']


In [169]:
for i in range(11):
    ttl=all_titles[i]
    ttl1=ttl+'1'
    all_data_v2[ttl1] = all_data_v2['Location'].str.contains(ttl)
    all_data_v2[ttl1] = all_data_v2[ttl1].map({True: 1, False: 0}) 
    
del all_data_v2['Location']

In [170]:
titles = list(all_data_v2['Fuel_Type'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  5


Unique Titles:
 ['CNG' 'DIESEL' 'PETROL' 'LPG' 'ELECTRIC']


In [171]:
for i in range(5):
    ttl=all_titles[i]
    ttl2=ttl+'2'
    all_data_v2[ttl2] = all_data_v2['Fuel_Type'].str.contains(ttl)
    all_data_v2[ttl2] = all_data_v2[ttl2].map({True: 1, False: 0})
    
del all_data_v2['Fuel_Type']

In [172]:
titles = list(all_data_v2['Transmission'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  2


Unique Titles:
 ['MANUAL' 'AUTOMATIC']


In [173]:
for i in range(2):
    ttl=all_titles[i]
    ttl2=ttl+'3'
    all_data_v2[ttl2] = all_data_v2['Transmission'].str.contains(ttl)
    all_data_v2[ttl2] = all_data_v2[ttl2].map({True: 1, False: 0})
    
del all_data_v2['Transmission']

In [174]:
titles = list(all_data_v2['Owner_Type'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  4


Unique Titles:
 ['FIRST' 'SECOND' 'FOURTH & ABOVE' 'THIRD']


In [175]:
for i in range(4):
    ttl=all_titles[i]
    ttl2=ttl+'4'
    all_data_v2[ttl2] = all_data_v2['Owner_Type'].str.contains(ttl)
    all_data_v2[ttl2] = all_data_v2[ttl2].map({True: 1, False: 0})
    
del all_data_v2['Owner_Type']

In [176]:
titles = list(all_data_v2['Year'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  23


Unique Titles:
 ['2010' '2015' '2011' '2012' '2013' '2016' '2018' '2014' '2017' '2007'
 '2009' '2008' '2019' '2006' '2005' '2004' '2002' '2000' '2003' '1999'
 '2001' '1998' '1996']


In [177]:
for i in range(23):
    ttl=all_titles[i]
    ttl2=ttl+'_5'
    all_data_v2[ttl2] = all_data_v2['Year'].str.contains(ttl)
    all_data_v2[ttl2] = all_data_v2[ttl2].map({True: 1, False: 0})
    
del all_data_v2['Year']

In [178]:
titles = list(all_data_v2['Brand'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  32


Unique Titles:
 ['MARUTI' 'HYUNDAI' 'HONDA' 'AUDI' 'NISSAN' 'TOYOTA' 'VOLKSWAGEN' 'TATA'
 'LAND' 'MITSUBISHI' 'RENAULT' 'MERCEDES-BENZ' 'BMW' 'MAHINDRA' 'FORD'
 'PORSCHE' 'DATSUN' 'JAGUAR' 'VOLVO' 'CHEVROLET' 'SKODA' 'MINI' 'FIAT'
 'JEEP' 'SMART' 'AMBASSADOR' 'ISUZU' 'FORCE' 'BENTLEY' 'LAMBORGHINI'
 'HINDUSTAN' 'OPELCORSA']


In [179]:
for i in range(32):
    ttl=all_titles[i]
    ttl2=ttl+'_6'
    all_data_v2[ttl2] = all_data_v2['Brand'].str.contains(ttl)
    all_data_v2[ttl2] = all_data_v2[ttl2].map({True: 1, False: 0})
    
del all_data_v2['Brand']

In [180]:
titles = list(all_data_v2['Sub_Brand'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  218


Unique Titles:
 ['WAGON' 'CRETA' 'JAZZ' 'ERTIGA' 'A4' 'EON' 'MICRA' 'INNOVA' 'VENTO'
 'INDICA' 'CIAZ' 'CITY' 'SWIFT' 'ROVER' 'PAJERO' 'AMAZE' 'DUSTER' 'NEW'
 '3' 'S' 'A6' 'I20' 'ALTO' 'WRV' 'COROLLA' 'SSANGYONG' 'VITARA' 'KUV'
 'M-CLASS' 'POLO' 'NANO' 'ELANTRA' 'XCENT' 'THAR' 'GRAND' 'KWID' 'I10'
 'X-TRAIL' 'ZEN' 'FIGO' 'C-CLASS' 'CAYENNE' 'XUV500' 'TERRANO' 'BRIO'
 'FIESTA' 'SANTRO' 'ZEST' 'RITZ' '5' 'FORTUNER' 'ECOSPORT' 'VERNA' 'GO'
 'OMNI' 'ETIOS' 'XF' 'EECO' 'CIVIC' 'V40' 'B' 'SCORPIO' 'CR-V' 'SLC' '1'
 'BEAT' 'RAPID' 'RS5' 'SUPERB' 'X5' 'GLC' 'COUNTRYMAN' 'OPTRA' 'LODGY'
 'E-CLASS' 'BALENO' 'LAURA' 'NUVOSPORT' 'FABIA' 'INDIGO' 'Q3' 'OCTAVIA'
 'A8' 'VERITO' 'COOPER' 'SANTA' 'X1' 'ACCENT' 'TUCSON' 'GLE' 'A-STAR'
 'GRANDE' 'X3' 'Q7' 'JETTA' 'GLA' 'CELERIO' 'SUMO' 'ACCORD' '6' 'MANZA'
 'SPARK' 'CLUBMAN' 'TEANA' '800' 'BRV' 'XE' 'XENON' 'A3' 'GL-CLASS' 'BR-V'
 'S80' 'CAPTUR' 'ENJOY' 'BOLERO' 'CEDIA' 'S-CROSS' 'YETI' 'E

In [181]:
for i in range(218):
    ttl=all_titles[i]
    ttl2=ttl+'_6'
    all_data_v2[ttl2] = all_data_v2['Sub_Brand'].str.contains(ttl)
    all_data_v2[ttl2] = all_data_v2[ttl2].map({True: 1, False: 0})
    
del all_data_v2['Sub_Brand']

In [182]:
all_data_v2.columns

Index(['Name', 'Kilometers_Driven', 'Final_Price', 'History', 'Seats_v1',
       'Power_v1', 'Engine_v1', 'Mileage_v1', 'MUMBAI1', 'PUNE1',
       ...
       'WR-V_6', 'CONTINENTAL_6', 'GALLARDO_6', 'F_6', 'MOTORS_6', 'FLYING_6',
       'MU_6', '370Z_6', 'ABARTH_6', '1.4GSI_6'],
      dtype='object', length=300)

In [183]:
all_data_v3.columns

Index(['Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type',
       'Transmission', 'Owner_Type', 'Brand', 'Sub_Brand', 'Final_Price',
       'History', 'Seats_v1', 'Power_v1', 'Engine_v1', 'Mileage_v1'],
      dtype='object')

In [184]:
all_data_v4=all_data_v3[['Name', 'Location', 'Year', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Brand', 'Sub_Brand']]

del all_data_v2['Name']
#del all_data_v2['Sub_Brand']

all_data_v5=pd.concat([all_data_v2,all_data_v4], axis=1)

In [185]:
train1=all_data_v5[:6001]
test1=all_data_v5[6001:7235]

In [186]:
#Feature Selection Step:
model1=xgb.XGBRegressor(colsample_bylevel=0.95, 
                       colsample_bytree=0.95,
                       gamma=1, #1
                       learning_rate=0.1, #0.1
                       max_depth=15,#14
                       min_child_weight=1, 
                       n_estimators=600, #500
                       objective='reg:linear', 
                       reg_alpha=1, 
                       reg_lambda=1,
                       scale_pos_weight=1, 
                       seed=99, 
                       subsample=0.95,
                       silent=True)
model1.fit(train1, target)

t1=model1.feature_importances_
t1=pd.DataFrame(t1)

t2=train1.columns
t2=pd.DataFrame(t2)

t1.columns=['Importance']
t2.columns=['Variable']

t3=pd.concat([t2,t1],axis=1)
t4=t3[t3['Importance'] > 0]

my_cols = list(t4.Variable)
all_data_new_v1 = all_data_v5[my_cols]
all_data_new_v1.head()

train1=all_data_new_v1[:6001]
test1=all_data_new_v1[6001:7235]

In [187]:
from sklearn.metrics import mean_squared_log_error
import math
def rmsle(real, predicted):
    real=np.expm1(real)
    predicted=np.expm1(predicted)
    return np.sqrt(mean_squared_log_error(real,predicted))
    
def rmsle_lgb(labels, preds):
    return 'rmsle', rmsle(preds,labels)

In [248]:
kf = StratifiedKFold(n_splits=30, shuffle=True, random_state=1234) #30
predicts = []
predicts_train = []
errxgb=[]
for train_index, test_index in kf.split(train1, target):
    print("###")
    X_train, X_val = train1.iloc[train_index], train1.iloc[test_index]
    y_train, y_val = target.iloc[train_index], target.iloc[test_index]
    xgb_params = {'n_estimators': 30000,
                  'n_jobs': -1,
                  'learning_rate':0.05, #0.05
                  'random_state':1234, 
                  'max_depth':7, #Done:7
                  'reg_alpha':1,
                  'colsample_bytree':0.1, #Done:0.1
                  'subsample':1, #1
                  'seed':1234 
                  }
    clf=XGBRegressor(**xgb_params)
    clf.fit(X_train, y_train,eval_set=[(X_val, y_val)],verbose=200,eval_metric='rmse',early_stopping_rounds=600)
    
    pr=clf.predict(X_val)
    print("errxgb: ",rmsle_lgb(y_val.values,pr)[1])    
    errxgb.append(rmsle_lgb(y_val.values,pr)[1])
    
    predicts_train.append(clf.predict(train1))
    predicts.append(clf.predict(test1))

C:\Users\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=30.
  % (min_groups, self.n_splits)), Warning)


###
[0]	validation_0-rmse:1.46843
Will train until validation_0-rmse hasn't improved in 600 rounds.
[200]	validation_0-rmse:0.159559
[400]	validation_0-rmse:0.145732
[600]	validation_0-rmse:0.142043
[800]	validation_0-rmse:0.140366
[1000]	validation_0-rmse:0.138808
[1200]	validation_0-rmse:0.137813
[1400]	validation_0-rmse:0.13724
[1600]	validation_0-rmse:0.137027
[1800]	validation_0-rmse:0.136662
[2000]	validation_0-rmse:0.136868
[2200]	validation_0-rmse:0.136838
[2400]	validation_0-rmse:0.136783
Stopping. Best iteration:
[1835]	validation_0-rmse:0.136613

errxgb:  0.13661279023753073
###
[0]	validation_0-rmse:1.38013
Will train until validation_0-rmse hasn't improved in 600 rounds.
[200]	validation_0-rmse:0.159703
[400]	validation_0-rmse:0.144504
[600]	validation_0-rmse:0.141998
[800]	validation_0-rmse:0.140628
[1000]	validation_0-rmse:0.139375
[1200]	validation_0-rmse:0.138802
[1400]	validation_0-rmse:0.138246
[1600]	validation_0-rmse:0.137785
[1800]	validation_0-rmse:0.137778
[2000

[3200]	validation_0-rmse:0.123384
[3400]	validation_0-rmse:0.123384
Stopping. Best iteration:
[2968]	validation_0-rmse:0.123371

errxgb:  0.12337054876064653
###
[0]	validation_0-rmse:2.06325
Will train until validation_0-rmse hasn't improved in 600 rounds.
[200]	validation_0-rmse:0.24714
[400]	validation_0-rmse:0.220103
[600]	validation_0-rmse:0.211218
[800]	validation_0-rmse:0.207046
[1000]	validation_0-rmse:0.204701
[1200]	validation_0-rmse:0.203143
[1400]	validation_0-rmse:0.202148
[1600]	validation_0-rmse:0.201364
[1800]	validation_0-rmse:0.200821
[2000]	validation_0-rmse:0.200409
[2200]	validation_0-rmse:0.200365
[2400]	validation_0-rmse:0.200365
[2600]	validation_0-rmse:0.200365
Stopping. Best iteration:
[2005]	validation_0-rmse:0.200365

errxgb:  0.20036505221216672
###
[0]	validation_0-rmse:1.62944
Will train until validation_0-rmse hasn't improved in 600 rounds.
[200]	validation_0-rmse:0.153421
[400]	validation_0-rmse:0.13316
[600]	validation_0-rmse:0.126783
[800]	validation_

[1800]	validation_0-rmse:0.124537
[2000]	validation_0-rmse:0.123796
[2200]	validation_0-rmse:0.123802
[2400]	validation_0-rmse:0.12384
[2600]	validation_0-rmse:0.123542
[2800]	validation_0-rmse:0.123248
[3000]	validation_0-rmse:0.123182
[3200]	validation_0-rmse:0.123108
[3400]	validation_0-rmse:0.123108
[3600]	validation_0-rmse:0.123108
Stopping. Best iteration:
[3014]	validation_0-rmse:0.123067

errxgb:  0.12306641451523896
###
[0]	validation_0-rmse:1.18199
Will train until validation_0-rmse hasn't improved in 600 rounds.
[200]	validation_0-rmse:0.130957
[400]	validation_0-rmse:0.114496
[600]	validation_0-rmse:0.110321
[800]	validation_0-rmse:0.109381
[1000]	validation_0-rmse:0.108768
[1200]	validation_0-rmse:0.107996
[1400]	validation_0-rmse:0.107295
[1600]	validation_0-rmse:0.107104
[1800]	validation_0-rmse:0.107066
[2000]	validation_0-rmse:0.107108
[2200]	validation_0-rmse:0.107118
[2400]	validation_0-rmse:0.106842
[2600]	validation_0-rmse:0.106818
[2800]	validation_0-rmse:0.106766

In [249]:
np.mean(errxgb)

0.1284715885769709

In [250]:
s=pd.DataFrame({'Price':np.expm1(np.mean(predicts,0))}) #max, mean, min, median
s.to_excel('xgb_impute_v3_mean.xlsx', index=False)#0.9438

In [251]:
s=pd.DataFrame({'Price':np.expm1(np.min(predicts,0))}) #max, mean, min, median
s.to_excel('xgb_impute_v3_min.xlsx', index=False)#0.9367

In [252]:
s=pd.DataFrame({'Price':np.expm1(np.max(predicts,0))}) #max, mean, min, median
s.to_excel('xgb_impute_v3_max.xlsx', index=False)#0.9410

In [253]:
s=pd.DataFrame({'Price':np.expm1(np.median(predicts,0))}) #max, mean, min, median
s.to_excel('xgb_impute_v3_median.xlsx', index=False)#0.9440

In [254]:
# For Stacking:
s=pd.DataFrame({'xgb_Price':(np.median(predicts_train,0))}) #max, mean, min
chk1=tg[['Price1']]
s1=pd.concat([s,chk1], axis=1)
s1.to_excel('xgb_train.xlsx', index=False)

s=pd.DataFrame({'xgb_Price':(np.median(predicts,0))}) #max, mean, min
s.to_excel('xgb_test.xlsx', index=False)

In [238]:
#Subsample:
#0.5:0.9438
#0.9:0.9438
#0.6:0.9438
#0.8:0.9439
#0.4:0.9434
#0.7:0.9439




#CV:
#30:0.9423
#5:0.9416
#20:0.9421
#10:0.9421
#15:0.9421
#25:0.9421

In [239]:
#colsample_bytree:
#1:0.9385
#0.9:0.9394
#0.8:0.9405
#0.7:0.9406
#0.6:0.9405
#0.5:0.9402
#0.4:0.9417
#0.3:0.9425
#0.2:0.9425
#0.1:0.9440
#0.07:0.9428
#0.125:0.9437

#12:0.9436
#11:0.9436
#10:0.9437
#9:0.9439





#0.3:0.9423
#0.2:0.9423
#0.4:0.9417
#0.1:0.9435

In [198]:
#Depth:
#0.1,8:0.9435
#0.1,4:0.9428
#0.1,7:0.9438
#0.1,5:0.9431
#0.1,6:0.9435

In [199]:
#Learning Rate:
#0.1,7,0.05:0.9438
#0.1,7,0.1:0.9432
#0.1,7,0.2:0.9422
#0.1,7,0.3:0.9412

In [200]:
#CV:
#20:0.9437
#5:0.9428
#15:0.9437
#25:0.9437
#10:0.9434
#35:0.9436

In [201]:
#Outlier removal colsample:
#0.1,30:0.9432
#0.1,15:0.9431
#0.1,25:0.9431
#0.1,10:0.9429
#0.1,20:0.9432

#0.2,30:0.9424
#0.3,30:0.9422

#0.1,15:0.9431
#0.2,15:0.9425
#0.3,15:0.9421

#0.1,25:0.9431

#30,4,0.1:0.9419
#30,6,0.1:0.9430
#30,5,0.1:

#0.1,7,10:0.9429
#0.1,7,20:0.9432